In [1]:
import os
import pandas as pd
from PIL import Image



import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader


import sys
from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from time import time

import torch
import torch.utils.data as data

import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize

import matplotlib.pyplot as plt
import seaborn as sns

In [6]:
data_dir = '/opt/ml/input/data/train'
img_dir = f'{data_dir}/crop_images'
df_path = f'{data_dir}/train.csv'

In [7]:
df = pd.read_csv(df_path)
df.head()

,id,gender,race,age,path
0,000001,female,Asian,45,000001_female_Asian_45
1,000002,female,Asian,52,000002_female_Asian_52
2,000004,male,Asian,54,000004_male_Asian_54
3,000005,female,Asian,58,000005_female_Asian_58
4,000006,female,Asian,59,000006_female_Asian_59


In [8]:
train_data_v1 = pd.read_csv("/opt/ml/train_with_label2.csv")
train_data_v1

,Unnamed: 0,gender,age,path,name,label
0,0,female,45,./input/data/train/images/000001_female_Asian_...,normal.jpg,16
1,1,female,45,./input/data/train/images/000001_female_Asian_...,incorrect_mask.jpg,10
2,2,female,45,./input/data/train/images/000001_female_Asian_...,mask3.jpg,4
3,3,female,45,./input/data/train/images/000001_female_Asian_...,mask4.jpg,4
4,4,female,45,./input/data/train/images/000001_female_Asian_...,mask5.jpg,4
...,...,...,...,...,...,...
18895,18895,male,19,./input/data/train/images/006959_male_Asian_19...,mask3.jpg,0
18896,18896,male,19,./input/data/train/images/006959_male_Asian_19...,mask4.jpg,0
18897,18897,male,19,./input/data/train/images/006959_male_Asian_19...,mask5.jpg,0
18898,18898,male,19,./input/data/train/images/006959_male_Asian_19...,mask2.jpg,0


In [9]:
img_path = train_data_v1['path']
labels = train_data_v1['label']

In [10]:
class TrainDataset(Dataset):
    def __init__(self, img_path, labels, transfrom):
        self.img_path = img_path
        self.labels = labels
        self.transform = transfrom

    def __getitem__(self,idx):
        x = Image.open(self.img_path[idx])
        y = self.labels[idx]

        if self.transform:
            x = self.transform(x)

        return x,y

    def __len__(self):
        return len(self.img_path)

In [11]:
x_train, x_valid, y_train, y_valid = train_test_split(img_path, labels, test_size=0.2)
x_train = x_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
x_valid = x_valid.reset_index(drop=True)
y_valid = y_valid.reset_index(drop=True)

In [12]:
train_transform = transforms.Compose([
    # transforms.CenterCrop(400),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomRotation(rotate_limit=15),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.220])
    ])

vaild_transform = transforms.Compose([
    # transforms.CenterCrop(380),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.220])
    ])

mask_train_dataset = TrainDataset(x_train, y_train, train_transform)
mask_valid_dataset = TrainDataset(x_valid, y_valid, vaild_transform)


In [13]:
from multiprocessing import cpu_count

num_workers = int(cpu_count()/2)

batch_size = 32

mask_train_dataloader = DataLoader(
    mask_train_dataset,
    batch_size = batch_size,
    shuffle=True,
    drop_last=True,
    num_workers=num_workers
    )

mask_valid_dataloader = DataLoader(
    mask_valid_dataset,
    batch_size=batch_size,
    )

In [14]:
# from efficientnet_pytorch import EfficientNet
# model = EfficientNet.from_pretrained('efficientnet-b4')


from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b4')

Loaded pretrained weights for efficientnet-b4


In [11]:
# class FocalLoss(nn.Module):
#     def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma
#         self.logits = logits
#         self.reduce = reduce

#     def forward(self, inputs, targets):
        
#         nn.CrossEntropyLoss()
    
#         ce_loss = nn.CrossEntropyLoss()(inputs, targets)

#         pt = torch.exp(-ce_loss)
#         F_loss = self.alpha * (1-pt)**self.gamma * ce_loss

#         if self.reduce:
#             return torch.mean(F_loss)
#         else:
#             return F_loss

In [15]:
class LabelSmoothingLoss(nn.Module):
    def __init__(self, classes, smoothing=0.0, dim=-1, weight = None):
        """if smoothing == 0, it's one-hot method
           if 0 < smoothing < 1, it's smooth method
        """
        super(LabelSmoothingLoss, self).__init__()
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.weight = weight
        self.cls = classes
        self.dim = dim

    def forward(self, pred, target):
        assert 0 <= self.smoothing < 1
        pred = pred.log_softmax(dim=self.dim)

        if self.weight is not None:
            pred = pred * self.weight.unsqueeze(0)   

        with torch.no_grad():
            true_dist = torch.zeros_like(pred)
            true_dist.fill_(self.smoothing / (self.cls - 1))
            true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        return torch.mean(torch.sum(-true_dist * pred, dim=self.dim))

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

print(f"{device} is using")

model.to(device)

LEARNING_RATE = 0.0001
NUM_EPOCH = 20

loss_fn = torch.nn.CrossEntropyLoss()
# loss_fn = LabelSmoothingLoss(classes=18)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

dataloaders = {
    "train" : mask_train_dataloader,
    "valid" : mask_valid_dataloader
}


cuda:0 is using


In [14]:
best_test_accuracy = 0.
best_test_loss = 0.

for epoch in range(NUM_EPOCH):
  n_iter = 0
  epoch_f1 = 0
  for phase in ["train", "valid"]:
    running_loss = 0.
    running_acc = 0.
    running_f1 = 0.
    if phase == "train":
      model.train() # 네트워크 모델을 train 모드로 두어 gradient을 계산하고, 여러 sub module (배치 정규화, 드롭아웃 등)이 train mode로 작동할 수 있도록 함
    elif phase == "valid":
      model.eval() # 네트워크 모델을 eval 모드 두어 여러 sub module들이 eval mode로 작동할 수 있게 함
  
    for ind, (images, labels) in enumerate(tqdm(dataloaders[phase])):
      images = images.to(device)
      labels = labels.to(device)

      optimizer.zero_grad() # parameter gradient를 업데이트 전 초기화함

      with torch.set_grad_enabled(phase == "train"): # train 모드일 시에는 gradient를 계산하고, 아닐 때는 gradient를 계산하지 않아 연산량 최소화
        logits = model(images)
        _, preds = torch.max(logits, 1) # 모델에서 linear 값으로 나오는 예측 값 ([0.9,1.2, 3.2,0.1,-0.1,...])을 최대 output index를 찾아 예측 레이블([2])로 변경함  
        loss = loss_fn(logits, labels)

        if phase == "train":
          loss.backward() # 모델의 예측 값과 실제 값의 CrossEntropy 차이를 통해 gradient 계산
          optimizer.step() # 계산된 gradient를 가지고 모델 업데이트
        
        epoch_f1 += f1_score(labels.cpu().numpy(), preds.cpu().numpy(), average='macro')
        n_iter += 1


      running_loss += loss.item() * images.size(0) # 한 Batch에서의 loss 값 저장
      running_acc += torch.sum(preds == labels.data) # 한 Batch에서의 Accuracy 값 저장
    #   running_f1 += f1_score(pred)
    # 한 epoch이 모두 종료되었을 때,
    epoch_loss = running_loss / len(dataloaders[phase].dataset)
    epoch_acc = running_acc / len(dataloaders[phase].dataset)
    epoch_f1 = epoch_f1/n_iter

    print(f"현재 epoch-{epoch}의 {phase}-데이터 셋에서 평균 Loss : {epoch_loss:.3f}, 평균 Accuracy : {epoch_acc:.3f}")
    if phase == "valid" and best_test_accuracy < epoch_acc: # phase가 test일 때, best accuracy 계산
      best_test_accuracy = epoch_acc
    if phase == "valid" and best_test_loss > epoch_loss: # phase가 test일 때, best loss 계산
      best_test_loss = epoch_loss
      torch.save(model,'/opt/ml/best_model_b7_CE_epo18.pth')
print("학습 종료!")
print(f"최고 accuracy : {best_test_accuracy}, 최고 낮은 loss : {best_test_loss}")

KeyboardInterrupt: 

In [ ]:
torch.save(model,'/opt/ml/best_model_smoothing+epo18.pth')

In [ ]:
zz = 3
print(zz)

3
